<a href="https://colab.research.google.com/github/DarkOxygen123/SBU-Hackathon/blob/main/Book1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Datasets to use**

**IEEE-CIS Fraud Detection**
**Fraudulent Transactions Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Loading Train Dataset

train_transaction = pd.read_csv('/content/drive/MyDrive/Datasets/dataset/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/content/drive/MyDrive/Datasets/dataset/ieee-fraud-detection/train_identity.csv')

# Loading Test Dataset

test_transaction = pd.read_csv('/content/drive/MyDrive/Datasets/dataset/ieee-fraud-detection/test_transaction.csv')
test_identity = pd.read_csv('/content/drive/MyDrive/Datasets/dataset/ieee-fraud-detection/test_identity.csv')

In [ ]:
print(f'train_transaction shape: {train_transaction.shape}')
print(f'train_identity shape: {train_identity.shape}')

print(f'test_transaction shape: {test_transaction.shape}')
print(f'test_identity shape: {test_identity.shape}')

train_transaction shape: (590540, 394)
train_identity shape: (144233, 41)
test_transaction shape: (506691, 393)
test_identity shape: (141907, 41)


In [ ]:
#mergin the data files for train and testing
merged_train = pd.merge(train_transaction, train_identity, how='left', on='TransactionID')
merged_test = pd.merge(test_transaction, test_identity, how='left', on='TransactionID')

In [ ]:
print(f'Merged train dataset: {merged_train.shape}')
print(f'Merged test dataset: {merged_test.shape}')

Merged train dataset: (590540, 434)
Merged test dataset: (506691, 433)


In [ ]:
del train_transaction, train_identity
del test_transaction, test_identity

In [ ]:
merged_test.to_pickle('/content/drive/MyDrive/Datasets/dataset/merged_test.pkl')

_____________________

# **Dataset analysis**

### Description of transaction Sataset

**TransactionID:** This serves as the unique identifier for each transaction and is utilized as the foreign key in the Identity Table.

**isFraud:** A binary indicator (0 or 1) denoting whether a transaction is fraudulent or not.

**TransactionDT:** Represents the timedelta from a specified reference datetime, although it is not an actual timestamp.

**TransactionAMT:** Indicates the payment amount for the transaction, expressed in USD.

**ProductCD:** Refers to the Product Code associated with the transaction. (*Catergorical Feature*)

**card1 - card6:** Encompasses payment card information, including card type, card category, issue bank, country, and other relevant details.

**addr:** Stands for Address.(*addr1, addr2 Catergorical Feature*)

**dist:** Represents the distance.

**P_emaildomain:** Denotes the Purchaser's Email Domain.(*Catergorical Feature*)

**R_emaildomain:** Denotes the Receiver's Email Domain.(*Catergorical Feature*)

**C1-C14:** Pertains to counting, indicating factors such as the number of addresses associated with the payment card. The exact meaning is obfuscated. (*card1-card6 catergorical Feature*)

**D1-D15:** Represent timedelta, indicating the days between the current and previous transactions, among other time-related factors.

**M1-M9:** Refers to match criteria, including aspects like matching names on the card and address.

**Vxxx:** Refers to Vesta engineered rich features, incorporating ranking, counting, and other entity relations.

### Description of Identity Dataset

Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
They're collected by Vesta’s fraud protection system and digital security partners.

(The field names are masked and pairwise dictionary is not provided for privacy protection and contract agreement from the dataset provider).

# **Preprocessing**

From visual Inspection and metaData we find that

- ProductCD
- card1-card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1-M9
- DeviceType
- DeviceInfo
- id_12 - id_38

are categorical in nature



In [ ]:
categorical_columns = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5','card6', 'addr1', 'addr2',\
                       'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'DeviceType', 'DeviceInfo', 'id_12', 'id_13',
                       'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23','id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33',
                       'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

In [ ]:
''' checking information on catergorical data '''
for i in categorical_columns:
  print(f'Number of unique values in {i}: {len(merged_train[i].unique())}')

Number of unique values in ProductCD: 5
Number of unique values in card1: 13553
Number of unique values in card2: 501
Number of unique values in card3: 115
Number of unique values in card4: 5
Number of unique values in card5: 120
Number of unique values in card6: 5
Number of unique values in addr1: 333
Number of unique values in addr2: 75
Number of unique values in P_emaildomain: 60
Number of unique values in R_emaildomain: 61
Number of unique values in M1: 3
Number of unique values in M2: 3
Number of unique values in M3: 3
Number of unique values in M4: 4
Number of unique values in M5: 3
Number of unique values in M6: 3
Number of unique values in M7: 3
Number of unique values in M8: 3
Number of unique values in M9: 3
Number of unique values in DeviceType: 3
Number of unique values in DeviceInfo: 1787
Number of unique values in id_12: 3
Number of unique values in id_13: 55
Number of unique values in id_14: 26
Number of unique values in id_15: 4
Number of unique values in id_16: 3
Numbe

In [ ]:
## Finding missing data levels in the columns

def missing_info(column, df):
    na = df[column].isna()
    count = na.sum()
    total_count = df.shape[0]
    miss_prcnt = np.round((count / total_count) * 100, 3)

    return count, miss_prcnt

In [ ]:
def column_nan_func(df):
    columns_missing_info = [
        (column, *missing_info(column, df))
        for column in df.columns
    ]

    columns_missing_info = [info for info in columns_missing_info if info[1] > 0]

    column_names = ['Feature_Name', 'Missing_Count', 'Missing_Percentage']
    missing_info_df = pd.DataFrame(columns_missing_info, columns=column_names)

    return missing_info_df

In [ ]:
missing_train_df = column_nan_func(merged_train)
missing_train_df

,Feature_Name,Missing_Count,Missing_Percentage
0,card2,8933,1.513
1,card3,1565,0.265
2,card4,1577,0.267
3,card5,4259,0.721
4,card6,1571,0.266
5,addr1,65706,11.126
6,addr2,65706,11.126
7,dist1,352271,59.652
8,dist2,552913,93.628
9,P_emaildomain,94456,15.995


Since we know that TransactionDT: Represents the timedelta from a specified reference datetime. We sorted the dataset according to this column. (Hoping that the time of of transaction has some significance to the occurances of fraudity)

In [ ]:
def sort_dataframe(df, sort_column, ascending=True):
    df.sort_values(by=sort_column, ascending=ascending, inplace=True)

sort_dataframe(merged_train, 'TransactionDT')

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0

Storing the current dataset to save memory for further processes

In [ ]:
merged_train.to_pickle('/content/drive/MyDrive/Datasets/dataset/merge_train.pkl')

In [ ]:
del merged_train

In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
model = TabNetClassifier()

In [ ]:
model

TabNetClassifier(n_d=8, n_a=8, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=None, output_dim=None, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[])